In [1]:
import pretty_midi
import glob
import pickle

# from keras.layers import Lambda, Input, Dense
# from keras.models import Model
# from keras.losses import mse, binary_crossentropy
# from keras.utils import plot_model
# from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import matplotlib.cm as cm
import PIL

In [2]:
def get_filename(filepath):
    base = os.path.basename(filepath)
    return os.path.splitext(base)[0]

In [3]:
NPITCH = 128
# output_t_size = 500 = 5 * 100 bc. 100 is default freq pretty_midi
def export_piano_rolls(roll, song_filename, output_t_size=500):
    pitch, ntime = roll.shape
    if pitch != NPITCH:
        print("Mismatch number of pitches")
    
    # Step half of size to train a connection between two pieces
    for st in range(0, ntime, output_t_size//2):
        roll_img = roll[:,st:st+output_t_size]
        if st + output_t_size > ntime:
            break
        dirpath = 'trainings/rolls_rgb/{}/'.format(song_filename)
        if st == 0 and not os.path.exists(dirpath):
            os.makedirs(dirpath)
        img_path = '{}st-{}.png'.format(dirpath, st)
#         print(roll_img[roll_img>0])
#       # First version rgb
#         plt.imsave(img_path, roll_img, cmap=cm.gray)
#       # First version gray
#         result = PIL.Image.fromarray((roll_img).astype(np.uint8), mode='L')

#       # Second version rgb
#         print(roll_img.shape)
        roll3D = np.zeros((3, 128, 128))
        roll3D[0][:] = roll_img
        roll3D[1][:] = roll_img
        roll3D[2][:] = roll_img
        roll3D = np.ascontiguousarray(roll3D.transpose(1,2,0))
#         print(roll3D.shape)
#         print(roll3D[roll3D>0])
        result = PIL.Image.fromarray(roll3D.astype(np.uint8), mode='RGB')
#         print(result)
    
        result.save(img_path)
#       # Load and check image
#         img = PIL.Image.open(img_path)
#         npimg = np.array(img)
#         print(npimg[npimg>0])
#         img = img.convert(mode="L")

In [4]:
def get_piano_rolls():
    pr = []
    for file in glob.glob("../type0/alb*.midi"):
        pm = pretty_midi.PrettyMIDI(file) #takes a midi file and converts to pretty_midi
        instr = pm.instruments #splits into list of instruments
        print("Processing ", file)
        for instrument in instr:
            name = pretty_midi.program_to_instrument_name(instrument.program)
            if name == "Acoustic Grand Piano": #only take the piano track
                piano_roll = instrument.get_piano_roll() #get the piano roll, which is a np.ndarray
                pr.append(piano_roll)  
                roll = piano_roll.astype(int)
#                 roll = (piano_roll[:,:]>0).astype(int)
                filename = get_filename(file)
                export_piano_rolls(roll, filename, output_t_size=128)
    return pr #list of piano rolls (np.ndarray matrices)

In [5]:
rolls = get_piano_rolls()
roll = rolls[0]

Processing  ../type0/alb_esp6_format0.midi
Processing  ../type0/alb_esp1_format0.midi
Processing  ../type0/alb_se7_format0.midi
Processing  ../type0/alb_se1_format0.midi
Processing  ../type0/alb_se6_format0.midi
Processing  ../type0/alb_se4_format0.midi
Processing  ../type0/alb_se3_format0.midi
Processing  ../type0/alb_se2_format0.midi
Processing  ../type0/alb_se5_format0.midi
Processing  ../type0/alb_esp3_format0.midi
Processing  ../type0/alb_esp4_format0.midi
Processing  ../type0/alb_se8_format0.midi
Processing  ../type0/alb_esp5_format0.midi
Processing  ../type0/alb_esp2_format0.midi


In [6]:
# for i in range(128):
#     for j in range(128):
#         if roll[i][j] > 0:
#             print(i, j, roll[i][j])